In [1]:
import numpy as np
import pandas as pd
import keras
import keras.backend as K
from keras.optimizers import Adam
from keras.models import Sequential
from keras.utils import Sequence
from keras.layers import *


In [2]:
data = pd.read_csv("C:/Users/Mayuresh Chimankar/Desktop/Coding/RL Project/sudoko_dataset/sudoku.csv")
try:
	data = pd.DataFrame({"quizzes": data["puzzle"], "solutions": data["solution"]})
except:
	pass


In [3]:
class DataGenerator(Sequence):
	def __init__(self, df,batch_size = 16,subset = "train",shuffle = False, info={}):
		super().__init__()
		self.df = df
		self.batch_size = batch_size
		self.shuffle = shuffle
		self.subset = subset
		self.info = info

		self.on_epoch_end()

	def __len__(self):
		return int(np.floor(len(self.df)/self.batch_size))
	def on_epoch_end(self):
		self.indexes = np.arange(len(self.df))
		if self.shuffle==True:
			np.random.shuffle(self.indexes)

	def __getitem__(self,index):
		X = np.empty((self.batch_size, 9,9,1))
		y = np.empty((self.batch_size,81,1))
		indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
		for i,f in enumerate(self.df['quizzes'].iloc[indexes]):
			self.info[index*self.batch_size+i]=f
			X[i,] = (np.array(list(map(int,list(f)))).reshape((9,9,1))/9)-0.5
		if self.subset == 'train':
			for i,f in enumerate(self.df['solutions'].iloc[indexes]):
				self.info[index*self.batch_size+i]=f
				y[i,] = np.array(list(map(int,list(f)))).reshape((81,1)) - 1
		if self.subset == 'train': return X, y
		else: return X


In [4]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

model.add(Flatten())
model.add(Dense(81*9))
model.add(Reshape((-1, 9)))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 9, 9, 64)          640       
                                                                 
 batch_normalization (Batch  (None, 9, 9, 64)          256       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 64)          36928     
                                                                 
 batch_normalization_1 (Bat  (None, 9, 9, 64)          256       
 chNormalization)                                                
                                                                 
 conv2d_2 (Conv2D)           (None, 9, 9, 128)         8320      
                                                                 
 flatten (Flatten)           (None, 10368)             0

In [5]:
model.load_weights('C:/Users/Mayuresh Chimankar/Desktop/Coding/RL Project/best_weights.hdf5')


In [6]:
def solve_sudoku_with_nn(model, puzzle):
	# Preprocess the input Sudoku puzzle
	puzzle = puzzle.replace('\n', '').replace(' ', '')
	initial_board = np.array([int(j) for j in puzzle]).reshape((9, 9, 1))
	initial_board = (initial_board / 9) - 0.5

	while True:
		# Use the neural network to predict values for empty cells
		predictions = model.predict(initial_board.reshape((1, 9, 9, 1))).squeeze()
		pred = np.argmax(predictions, axis=1).reshape((9, 9)) + 1
		prob = np.around(np.max(predictions, axis=1).reshape((9, 9)), 2)

		initial_board = ((initial_board + 0.5) * 9).reshape((9, 9))
		mask = (initial_board == 0)

		if mask.sum() == 0:
			# Puzzle is solved
			break

		prob_new = prob * mask

		ind = np.argmax(prob_new)
		x, y = (ind // 9), (ind % 9)

		val = pred[x][y]
		initial_board[x][y] = val
		initial_board = (initial_board / 9) - 0.5

	# Convert the solved puzzle back to a string representation
	solved_puzzle = ''.join(map(str, initial_board.flatten().astype(int)))

	return solved_puzzle


In [8]:
def print_sudoku_grid(puzzle):
	puzzle = puzzle.replace('\n', '').replace(' ', '')
	for i in range(9):
		if i % 3 == 0 and i != 0:
			print("-"*21)

		for j in range(9):
			if j % 3 == 0 and j != 0:
				print("|", end=" ")
			print(puzzle[i*9 + j], end=" ")
		print()
new_game = '''
0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0
'''

game = '''
0 0 0 7 0 0 0 9 6
0 0 3 0 6 9 1 7 8
0 0 7 2 0 0 5 0 0
0 7 5 0 0 0 0 0 0
9 0 1 0 0 0 3 0 0
0 0 0 0 0 0 0 0 0
0 0 9 0 0 0 0 0 1
3 1 8 0 2 0 4 0 7
2 4 0 0 0 5 0 0 0
'''

solved_puzzle_nn = solve_sudoku_with_nn(model, game)

# Print the solved puzzle as a grid
print("Sudoku Solution (NN):")
print_sudoku_grid(solved_puzzle_nn)


1/1 [==============================] - 0s 31ms/step
Sudoku Solution (NN):
1 8 4 | 7 5 3 | 2 9 6 
5 2 3 | 4 6 9 | 1 7 8 
6 9 7 | 2 1 8 | 5 3 4 
---------------------
4 7 5 | 1 3 2 | 8 6 9 
9 6 1 | 5 8 7 | 3 4 2 
8 3 2 | 6 4 7 | 7 1 5 
---------------------
7 5 9 | 8 3 4 | 6 2 1 
3 1 8 | 9 2 6 | 4 5 7 
2 4 6 | 1 7 5 | 9 8 3 
